In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = (torch.device('cuda') if torch.cuda.is_available()
         else torch.device('cpu'))
print(f"training on device {device}")

training on device cuda


In [2]:
import datetime

def training_loop(n_epochs,optimizer,model,loss_fn,train_loader):
  for epoch in range(1,n_epochs+1):
    loss_train=0.0
    for imgs,labels in train_loader:
      imgs = imgs.to(device=device)
      labels = labels.to(device=device)
      outputs=model(imgs)
      loss=loss_fn(outputs,labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loss_train += loss.item()

    if epoch==1 or epoch %10==0:
      print('{} Epoch {}, Training loss {}'.format(
          datetime.datetime.now(),epoch,loss_train/len(train_loader)
      ))

In [4]:
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

trainset = torchvision.datasets.CIFAR10(root='./data',train=True,
                                        download=True,transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 4,
                                                shuffle=True,num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data',train=False,
                                        download=True,transform=transform)

classes = ('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')
label_map = {0:0, 2:1}
class_names = ['airplane','bird']
cifar2 = [(img,label_map[label]) for img,label in trainset if label in [0,2]]
cifar2_val = [(img,label_map[label]) for img,label in testset if label in [0,2]]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified



In [6]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1)
    #self.act1 = nn.Tanh()
    #self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8,kernel_size=3,padding=1)
    #self.act2 = nn.Tanh()
    #self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    #self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,2)

  def forward(self,x):
    out = F.max_pool2d(torch.tanh(self.conv1(x)),2)
    out = F.max_pool2d(torch.tanh(self.conv2(out)),2)
    out = out.view(-1,8*8*8)
    out = torch.tanh(self.fc1(out))
    out = self.fc2(out)
    return out

In [7]:
train_loader = torch.utils.data.DataLoader(cifar2,batch_size=64,shuffle=True)

model = Net().to(device=device)
optimizer=optim.SGD(model.parameters(),lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader
)

2020-11-07 09:36:34.070707 Epoch 1, Training loss 0.620808625297182
2020-11-07 09:36:37.418037 Epoch 10, Training loss 0.3758643646338943
2020-11-07 09:36:41.191284 Epoch 20, Training loss 0.32391722301009357
2020-11-07 09:36:44.878098 Epoch 30, Training loss 0.30273039866784573
2020-11-07 09:36:48.689548 Epoch 40, Training loss 0.28606613312557244
2020-11-07 09:36:52.434879 Epoch 50, Training loss 0.26344013437154185
2020-11-07 09:36:56.253194 Epoch 60, Training loss 0.24286759084766837
2020-11-07 09:36:59.925820 Epoch 70, Training loss 0.22597956320472584
2020-11-07 09:37:03.585148 Epoch 80, Training loss 0.21175817196156568
2020-11-07 09:37:07.271192 Epoch 90, Training loss 0.19840226137334374
2020-11-07 09:37:11.026549 Epoch 100, Training loss 0.1841834545799881
